In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
import warnings                   # To ignore the warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/final-data/train_final.csv
/kaggle/input/final-data/test_final.csv


In [2]:
# Reading the test data files
train_final_merged = pd.read_csv('/kaggle/input/final-data/train_final.csv')
test_final_merged = pd.read_csv('/kaggle/input/final-data/test_final.csv')

In [3]:
# checking the final merged test dataset
test_final_merged.head()

,Unnamed: 0,farm_id,ingredient_type,id,operations_commencing_year,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour
0,0,fid_110884,ing_w,0,new,7,690.455096,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
1,1,fid_90053,ing_w,1,new,7,252.696160,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
2,2,fid_17537,ing_w,2,moderate,7,499.446528,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
3,3,fid_110392,ing_w,3,moderate,7,2200.407555,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0
4,4,fid_62402,ing_w,4,moderate,7,10833.140121,Obery Farms,location 7369,17.8,4.0,100.0,1021.4,0.0,3.6,1,1,0


In [4]:
# checking the  data types of final merged tarin dataset
train_final_merged.dtypes

Unnamed: 0                      int64
farm_id                        object
ingredient_type                object
yield                         float64
operations_commencing_year     object
num_processing_plants           int64
farm_area                     float64
farming_company                object
deidentified_location          object
temp_obs                      float64
cloudiness                    float64
wind_direction                float64
pressure_sea_level            float64
precipitation                 float64
wind_speed                    float64
month                           int64
day                             int64
hour                            int64
dtype: object

In [5]:
train_ing_y = train_final_merged[train_final_merged['ingredient_type'] == 'ing_y']

In [6]:
# Split the dataset into three equal parts
train_ing_y_splits = np.array_split(train_ing_y, 3)

# Access and use the first split as a separate dataset
train_y = train_ing_y_splits[0]

In [7]:
test_ing_y = test_final_merged[test_final_merged['ingredient_type'] == 'ing_y']

In [8]:
# Split the dataset into three equal parts
test_ing_y_splits = np.array_split(test_ing_y, 3)

# Access and use the first split as a separate dataset
test_y = test_ing_y_splits[0]

In [9]:
# keep a copy of test w we need it in creating submission file
import copy
tes = copy.deepcopy(test_y)

* farm_id is unique just like my id. my performance is measured based on my skills on based on my id right! same here the prediction of yield is depend on loaction and weather condtions on id. so let's drop it

In [10]:
# dropping the farm_id and num_processing pants from test and train datasets
train_y.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)
test_y.drop(['farm_id','Unnamed: 0'], axis=1, inplace=True)

In [11]:
# assigning object columns in category columns
cat_cols=['ingredient_type','deidentified_location',
          'operations_commencing_year','farming_company']
cat_cols

['ingredient_type',
 'deidentified_location',
 'operations_commencing_year',
 'farming_company']

In [12]:
# converting object columns to category columns
train_y[cat_cols] = train_y[cat_cols].astype("category")
test_y[cat_cols] = test_y[cat_cols].astype("category")

# Model Building and Predictions

In [13]:
# deviding the data into x and y
from sklearn.model_selection import train_test_split
X = train_y.drop(['yield'] , axis = 1)
y = train_y['yield']

In [14]:
# splitting the data into train and test in the ration of 70% & 30%
X_train, X_test, y_train, y_test =   train_test_split(X, y, test_size=0.30,random_state = 123)

In [15]:
# checking the shapes
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(634628, 15) (271984, 15)
(634628,) (271984,)


<div class="alert alert-block alert-info"><b> 
  Dummifing categorical features into numerical using get dummies
</b></div> 

In [16]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
X_train.deidentified_location = label.fit_transform(X_train.deidentified_location)
X_test.deidentified_location = label.transform(X_test.deidentified_location)
X_train.farming_company = label.fit_transform(X_train.farming_company)
X_test.farming_company = label.transform(X_test.farming_company)

In [17]:
## Convert Categorical Columns to numerical
X_train = pd.get_dummies(X_train, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

X_test = pd.get_dummies(X_test, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [18]:
# scaling the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
num_cols = ["farm_area", "wind_speed", "precipitation", "temp_obs",
            "pressure_sea_level","wind_direction",'num_processing_plants']

# scale on train
X_train[num_cols] =scaler.fit_transform(X_train[num_cols])

# scale on test
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [19]:
# checking the x train data
X_train.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
6178601,0.0,0.778394,12,0,0.707646,2.0,0.805556,0.527825,0.004608,0.364706,5,29,14,1,0
6648579,0.0,0.330657,8,0,0.758621,2.0,0.083333,0.585160,0.004608,0.123529,7,30,7,1,0
5679493,0.0,0.778394,12,0,0.482759,0.0,0.000000,0.544688,0.004608,0.000000,3,25,4,1,0
6278017,0.0,0.102662,12,0,0.833583,2.0,0.527778,0.537943,0.004608,0.088235,6,11,15,1,0
6011694,0.0,0.668293,12,0,0.716642,2.0,0.944444,0.544688,0.004608,0.364706,5,7,17,1,0


In [20]:
# checking the x test data
X_test.head()

,num_processing_plants,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,pressure_sea_level,precipitation,wind_speed,month,day,hour,operations_commencing_year_new,operations_commencing_year_old
7628992,0.0,0.148899,12,0,0.482759,2.0,0.388889,0.404722,0.004608,0.517647,12,5,20,1,0
6303654,0.0,0.063334,8,0,0.742129,8.0,0.861111,0.401349,0.082949,0.335294,6,15,0,1,0
6597705,0.0,0.017940,11,0,0.841079,2.0,0.305556,0.487352,0.004608,0.335294,7,23,15,1,0
5852959,0.0,0.063334,8,0,0.824588,2.0,0.472222,0.652614,0.004608,0.576471,4,16,22,1,0
11915227,0.0,0.123982,12,1,0.358321,2.0,0.222222,0.430017,0.004608,0.123529,4,9,10,0,0


In [21]:
# checking the data types x_train data
X_train.dtypes

num_processing_plants             float64
farm_area                         float64
farming_company                     int64
deidentified_location               int64
temp_obs                          float64
cloudiness                        float64
wind_direction                    float64
pressure_sea_level                float64
precipitation                     float64
wind_speed                        float64
month                               int64
day                                 int64
hour                                int64
operations_commencing_year_new      uint8
operations_commencing_year_old      uint8
dtype: object

# Model Building

<div class="alert alert-block alert-info"><b> 
  Linear Regression
</b>

In [22]:
# linear regression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)
train_pred =reg.predict(X_train)
test_pred = reg.predict(X_test)
lr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
lr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('RMSE of Linear Regressor on train : ',lr_train_pred)
print('RMSE of Linear Regressor on test : ',lr_test_pred)

RMSE of Linear Regressor on train :  727642.326964791
RMSE of Linear Regressor on test :  701886.3530710585


<div class="alert alert-block alert-info"><b> 
  Random Forest Regressor
</b></div> 

In [23]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
train_pred= rfr.predict(X_train)
test_pred = rfr.predict(X_test)
rfr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
rfr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('RMSE of RandomForestRegressor on train', rfr_train_pred)
print('RMSE of RandomForestRegressor on test:', rfr_test_pred)

RMSE of RandomForestRegressor on train 51014.55068285992
RMSE of RandomForestRegressor on test: 121747.09179339005


<div class="alert alert-block alert-info"><b> 
  Gradient Boosting Regressor
</b></div> 

In [24]:
from sklearn.ensemble import GradientBoostingRegressor
gbg = GradientBoostingRegressor(learning_rate=0.05,n_estimators=200,random_state=0)
gbg.fit(X_train,y_train)
train_pred=gbg.predict(X_train)
test_pred = gbg.predict(X_test)
gbr_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
gbr_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('GBR on train : ', gbr_train_pred)
print('GBR on test : ',gbr_test_pred)

GBR on train :  400566.65744085965
GBR on test :  383842.7613115324


<div class="alert alert-block alert-info"><b> 
  XGB Regressor
</b></div> 

In [25]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators=500,learning_rate=0.01)
xgb.fit(X_train,y_train)
train_pred=xgb.predict(X_train)
test_pred = xgb.predict(X_test)
xgb_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
xgb_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('XGB Regressor on train : ', xgb_train_pred)
print('XGB Regressor on test : ',xgb_test_pred)

XGB Regressor on train :  217056.65387000985
XGB Regressor on test :  224605.4628931582


<div class="alert alert-block alert-info"><b> 
  Light Gradient Boosting Regressor
</b></div> 

In [26]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(X_train,y_train)
train_pred=lgb.predict(X_train)
test_pred = lgb.predict(X_test)
lgb_train_pred = np.sqrt(mean_squared_error(y_train, train_pred))
lgb_test_pred = np.sqrt(mean_squared_error(y_test, test_pred))
print('LGB Regressor on train:', lgb_train_pred)
print('LGB Regressor on test :',lgb_test_pred)

LGB Regressor on train: 118525.39867135262
LGB Regressor on test : 144469.22846026125


<div class="alert alert-block alert-info"><b> 
  Results of training and testing set
</b></div> 

In [27]:
# Creating a dataframe showing the RMSE for both training and testing data
results=pd.DataFrame({
                    'Models':['Linear Regression','GradientBoost','LGBMRegressor','Random Forest',
                              'XGBoost'],
    
                    'Train_RMSE':[lr_train_pred, gbr_train_pred, lgb_train_pred, rfr_train_pred,
                                  xgb_train_pred],
    
                    'Test_RMSE':[lr_test_pred, gbr_test_pred, lgb_test_pred, rfr_test_pred,
                                 xgb_test_pred],
                     })

In [28]:
results.sort_values('Test_RMSE',ascending=True,ignore_index=True)

,Models,Train_RMSE,Test_RMSE
0,Random Forest,51014.550683,121747.091793
1,LGBMRegressor,118525.398671,144469.228460
2,XGBoost,217056.653870,224605.462893
3,GradientBoost,400566.657441,383842.761312
4,Linear Regression,727642.326965,701886.353071


## Conclusion

<div class="alert alert-block alert-info"><b> 
   Light gradient regressor gave the best results as compared to all the other algorithms we tried.
</b></div> 

In [29]:
#RandomSearchCV designed to find the best parameters to improve our model
#from sklearn.model_selection import RandomizedSearchCV
#rf_grid = RandomForestRegressor()
#rf_param_grid = {"n_estimators" : [100,500,1000],
#                 "max_features" : ['sqrt','log2'],
#                 "max_depth" : [10,20,30],
#                "min_samples_leaf" : [2,3,5,7,10]}
#rf2=RandomizedSearchCV(rf_grid,rf_param_grid,cv=5,n_jobs=-1,n_iter=10)
#rf2.fit(X_train,y_train.values.ravel())
#best_hyper = rf2.best_params_

In [30]:
#best model
#best_model = RandomForestRegressor()
#best_model.fit(X_train,y_train)
#train_pred=rf2.predict(X_train)
#test_pred = rf2.predict(X_test)
#print('RMSE RandomizedSearchCV on train : ', np.sqrt(mean_squared_error(y_train, train_pred)))
#print('RMSE RandomizedSearchCV on test : ', np.sqrt(mean_squared_error(y_test, test_pred)))

## Lets work on test data

In [31]:
test_y.deidentified_location = label.fit_transform(test_y.deidentified_location)
test_y.farming_company = label.fit_transform(test_y.farming_company)

In [32]:
# converting categorical to numerical
test_y = pd.get_dummies(test_y, columns=['ingredient_type','operations_commencing_year'],drop_first=True)

In [33]:
# scale on test
test_y[num_cols] = scaler.fit_transform(test_y[num_cols])

In [34]:
# selecting remaing columns except id column
fin = test_y.loc[:, test_y.columns != 'id']

In [35]:
# checking the final test dataset dimensions
fin.shape

(949785, 15)

# Predictions

In [37]:
# predicting the test data with best model
test_pred = lgb.predict(fin)

In [38]:
# creating a dataframe with id column and yield column
sub  = pd.DataFrame({'id': tes.id, 'yield': test_pred})

In [39]:
# checking the shape of the final dataframe 
sub.shape

(949785, 2)

In [40]:
# checking the head of the dataframe
sub.head()

,id,yield
1138826,37198374,761.613040
1138920,37198624,761.613040
1139014,37198874,838.733587
1139108,37199124,838.733587
1139202,37199374,838.733587


In [41]:
# checking the last 5 rows of the dataframe
sub.tail()

,id,yield
12389991,37751673,1013.888264
12389995,37751677,466.104864
12389999,37751681,1804.723210
12390003,37751685,537.884188
12390008,37751690,1257.085801


In [42]:
# creating submission file
sub.to_csv('submission_y1.csv', index = None)